## 1.
Обучить любую модель классификации на датасете IRIS до применения PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке

In [136]:
import numpy as np

In [137]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import datasets

In [138]:
import random

In [139]:
def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [140]:
# Загрузим игрушечный датасет из sklearn
iris = datasets.load_iris()
X = iris.data
X.shape

(150, 4)

In [141]:
test_ind = random.sample(range(0,150), 150)

In [143]:
split_n = 40
X_test = X[test_ind[:split_n]]
X_train = X[test_ind[split_n:]]

In [144]:
Y = iris.target
Y_test = Y[test_ind[:split_n]]
Y_train = Y[test_ind[split_n:]]

In [145]:
Y_test

array([0, 1, 2, 1, 0, 1, 0, 1, 0, 1, 1, 2, 2, 2, 1, 0, 2, 0, 2, 0, 2, 2,
       0, 0, 2, 1, 0, 2, 0, 0, 1, 1, 1, 2, 0, 2, 2, 2, 2, 2])

In [146]:
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [147]:
model = DecisionTreeClassifier(max_depth=3, random_state=1)

In [148]:
n_objects = X_train.shape[0]
w = np.ones(n_objects) / n_objects

In [149]:
model.fit(X_train, Y_train, sample_weight=w)

pred = model.predict(X_test)

print(pred)

[0 2 2 1 0 1 0 1 0 1 1 2 2 2 1 0 2 0 2 0 2 2 0 0 2 1 0 2 0 0 1 1 1 2 0 2 2
 2 2 2]


In [150]:
ac_1 = accuracy(pred, Y_test)
print(ac_1)

0.975


In [151]:
# Для начала отмасштабируем выборку
X = X.astype(float)

X = standard_scale(X)

In [152]:
# Найдем собственные векторы и собственные значения
 
covariance_matrix = X.T @ X

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

In [153]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперь оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329986, 22.85076178670177, 3.668921889282877, 0.5178709107154922]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


In [154]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(2)])

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [155]:
# Сформируем новую матрицу "объекты-признаки"
Z = X.dot(W)

In [156]:
split_n = 40
Z_test = Z[test_ind[:split_n]]
Z_train = Z[test_ind[split_n:]]

In [157]:
model.fit(Z_train, Y_train, sample_weight=w)

pred_2 = model.predict(Z_test)

print(pred_2)

[0 2 2 1 0 1 0 1 0 1 1 2 1 1 1 0 2 0 2 0 2 2 0 0 2 1 0 1 0 0 1 1 1 2 0 1 2
 1 2 1]


In [158]:
ac_2 = accuracy(pred_2, Y_test)
print(ac_2)

0.825
